In [37]:
import requests
import pandas as pd

In [38]:
df20 = pd.read_csv("../sources/calendar/myhora-holiday-calendar-2563.csv")
df20.drop(columns=["Start Time","End Date", "End Time", "All day event", "Description", 'Show time as', 'Location'], inplace=True)

df21 = pd.read_csv("../sources/calendar/myhora-holiday-calendar-2564.csv")
df21.drop(columns=["Start Time","End Date", "End Time", "All day event", "Description", 'Show time as', 'Location'], inplace=True)

df22 = pd.read_csv("../sources/calendar/myhora-holiday-calendar-2565.csv")
df22.drop(columns=["Start Time","End Date", "End Time", "All day event", "Description", 'Show time as', 'Location'], inplace=True)

df23 = pd.read_csv("../sources/calendar/myhora-holiday-calendar-2566.csv")
df23.drop(columns=["Start Time","End Date", "End Time", "All day event", "Description", 'Show time as', 'Location'], inplace=True)

df24 = pd.read_csv("../sources/calendar/myhora-holiday-calendar-2567.csv")
df24.drop(columns=["Start Time","End Date", "End Time", "All day event", "Description", 'Show time as', 'Location'], inplace=True)

df25 = pd.read_csv("../sources/calendar/myhora-holiday-calendar-2568.csv")
df25.drop(columns=["Start Time","End Date", "End Time", "All day event", "Description", 'Show time as', 'Location'], inplace=True)

In [39]:
# Convert 'Start Date' columns from yyyymmdd (string/int) to datetime across all loaded DataFrames
for _df in [df20, df21, df22, df23, df24, df25]:
    _df['Start Date'] = pd.to_datetime(_df['Start Date'].astype(str), format='%Y%m%d')

In [40]:
# สร้างฟังก์ชันและเพิ่มวันเทศกาล (festival days) ลงในแต่ละ DataFrame (df20 - df25)
from datetime import datetime
import pandas as pd
import re

# ฟังก์ชันหา second Saturday ของเดือนมกราคม ของปีที่ระบุ
def second_saturday_of_jan(year: int) -> pd.Timestamp:
    d = pd.Timestamp(year=year, month=1, day=1)
    while d.weekday() != 5:  # 5 = Saturday
        d += pd.Timedelta(days=1)
    return d + pd.Timedelta(days=7)

# ฟังก์ชัน normalize ชื่อวันหยุด/เทศกาล ให้รูปแบบมาตรฐานเพื่อลดการซ้ำ
NORMALIZE_MAP = {
    'วันสงกรานต์': 'สงกรานต์',
    'สงกรานต์': 'สงกรานต์',
    'New Year Countdown': 'วันสิ้นปี',
    'คริสมาส': 'คริสต์มาส',
    'คริสมาสอีฟ': 'คริสต์มาสอีฟ',
}

def normalize_subject(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip()
    if 'สงกรานต์' in s:
        return 'สงกรานต์'
    return NORMALIZE_MAP.get(s, s)

# เทศกาลที่ต้องการให้เป็นประเภท festival (ยกเว้นสงกรานต์ = holiday)
FESTIVAL_SUBJECTS = {
    'วันเด็ก', 'วาเลนไทน์', 'คริสต์มาสอีฟ', 'คริสต์มาส', 'วันสิ้นปี'
}

# สร้าง DataFrame วันเทศกาลสำหรับปีใดปีหนึ่ง พร้อม Category
# Category: holiday / festival
def generate_festivals(year: int) -> pd.DataFrame:
    second_sat = second_saturday_of_jan(year)
    rows = [
        ["วันเด็ก", second_sat],
        ["วาเลนไทน์", f"{year}-02-14"],
        ["สงกรานต์", f"{year}-04-13"],
        ["สงกรานต์", f"{year}-04-14"],
        ["สงกรานต์", f"{year}-04-15"],
        ["คริสต์มาสอีฟ", f"{year}-12-24"],
        ["คริสต์มาส", f"{year}-12-25"],
        ["วันสิ้นปี", f"{year}-12-31"],
    ]
    fest_df = pd.DataFrame(rows, columns=["Subject", "Start Date"])
    fest_df["Start Date"] = pd.to_datetime(fest_df["Start Date"])  # second_sat already Timestamp
    fest_df['Subject'] = fest_df['Subject'].apply(normalize_subject)
    # กำหนด Category: สงกรานต์ = holiday (เพราะเป็นวันหยุดราชการ) อื่น ๆ = festival
    fest_df['Category'] = fest_df['Subject'].apply(lambda s: 'holiday' if s == 'สงกรานต์' else ('festival' if s in FESTIVAL_SUBJECTS else 'holiday'))
    return fest_df

# จับคู่ชื่อ DataFrame กับปีค.ศ. (ไฟล์ 2563-2568 -> 2020-2025)
df_map = {
    2020: df20,
    2021: df21,
    2022: df22,
    2023: df23,
    2024: df24,
    2025: df25,
}

# เพิ่มเทศกาล: normalize ก่อน แล้ว dedup โดย (Subject + Start Date) และเติม Category
for year, _df in df_map.items():
    # แปลงประเภทวันที่ให้ชัวร์
    if not pd.api.types.is_datetime64_any_dtype(_df['Start Date']):
        _df['Start Date'] = pd.to_datetime(_df['Start Date'])

    # เพิ่มคอลัมน์ Category ถ้ายังไม่มี -> ตั้งเป็น holiday (ข้อมูลเดิมถือว่าเป็นวันหยุดทางการ)
    if 'Category' not in _df.columns:
        _df['Category'] = 'holiday'

    # Normalize ชื่อเดิม
    _df['Subject'] = _df['Subject'].apply(normalize_subject)

    # ลบซ้ำรอบแรก
    _df = _df.drop_duplicates(subset=["Subject", "Start Date"], keep='first')

    fest = generate_festivals(year)
    existing_pairs = set(zip(_df['Subject'], _df['Start Date']))

    rows_to_add = []
    for _, row in fest.iterrows():
        subj = row['Subject']
        dt = row['Start Date']
        if (subj, dt) in existing_pairs:
            continue
        rows_to_add.append(row)

    if rows_to_add:
        add_df = pd.DataFrame(rows_to_add)
        _df = pd.concat([_df, add_df], ignore_index=True)

    # Normalize อีกครั้ง (ความปลอดภัย)
    _df['Subject'] = _df['Subject'].apply(normalize_subject)

    # กันกรณี festival ทับ holiday: ถ้ามีสัญญาณว่า Subject เดียววันเดียวกันแต่ Category ต่างกัน -> holiday ชนะ (ยกเว้นอยาก override ให้เปลี่ยนที่นี่)
    _df.sort_values(['Subject', 'Start Date', 'Category'], inplace=True)
    # ให้เรียงให้ holiday มาก่อน (เพราะจะ keep='first')
    _df['Category_rank'] = _df['Category'].apply(lambda c: 0 if c == 'holiday' else 1)
    _df.sort_values(['Subject', 'Start Date', 'Category_rank'], inplace=True)
    _df = _df.drop_duplicates(subset=['Subject', 'Start Date'], keep='first')
    _df.drop(columns=['Category_rank'], inplace=True)

    _df.sort_values('Start Date', inplace=True)
    _df.reset_index(drop=True, inplace=True)
    df_map[year] = _df

(df20, df21, df22, df23, df24, df25) = (df_map[2020], df_map[2021], df_map[2022], df_map[2023], df_map[2024], df_map[2025])

# ตรวจสอบจำนวนซ้ำหลัง normalize
duplicate_counts = {y: df_map[y].duplicated(subset=["Subject", "Start Date"]).sum() for y in df_map}
print("จำนวนแถวซ้ำ (ควรเป็น 0):", duplicate_counts)

# ตรวจสอบตัวอย่างวันสงกรานต์ปี 2023
print("สงกรานต์ 2023:")
print(df23[df23['Start Date'].between('2023-04-10','2023-04-20') & df23['Subject'].str.contains('สงกรานต์')][['Subject','Start Date','Category']])

# แสดงตัวอย่างปีล่าสุด
df25.tail(15)[['Subject','Start Date','Category']]

จำนวนแถวซ้ำ (ควรเป็น 0): {2020: 0, 2021: 0, 2022: 0, 2023: 0, 2024: 0, 2025: 0}
สงกรานต์ 2023:
    Subject Start Date Category
6  สงกรานต์ 2023-04-13  holiday
7  สงกรานต์ 2023-04-14  holiday
8  สงกรานต์ 2023-04-15  holiday
9  สงกรานต์ 2023-04-17  holiday


,Subject,Start Date,Category
15,วันหยุดชดเชย,2025-05-12,holiday
16,วันหยุดพิเศษ (ครม.),2025-06-02,holiday
17,วันเฉลิมฯ พระบรมราชินี,2025-06-03,holiday
18,วันอาสาฬหบูชา,2025-07-10,holiday
19,วันเข้าพรรษา (ราชการ),2025-07-11,holiday
20,วันเฉลิมฯ พระวชิรเกล้าเจ้าอยู่หัว,2025-07-28,holiday
21,วันหยุดพิเศษ (ครม.),2025-08-11,holiday
22,วันแม่,2025-08-12,holiday
23,วันนวมินทรมหาราช,2025-10-13,holiday
24,วันปิยมหาราช,2025-10-23,holiday


In [41]:
df25

,Subject,Start Date,Category
0,วันขึ้นปีใหม่,2025-01-01,holiday
1,วันเด็ก,2025-01-11,festival
2,วันมาฆบูชา,2025-02-12,holiday
3,วาเลนไทน์,2025-02-14,festival
4,วันจักรี,2025-04-06,holiday
5,วันหยุดชดเชย,2025-04-07,holiday
6,สงกรานต์,2025-04-13,holiday
7,สงกรานต์,2025-04-14,holiday
8,สงกรานต์,2025-04-15,holiday
9,วันหยุดชดเชย,2025-04-16,holiday


In [43]:
special_days = pd.concat([df20, df21, df22, df23, df24, df25], ignore_index=True)
special_days.to_csv("../sources/calendar/special_days_2020_2025.csv", index=False) 